---

# CSCI 3202, Fall 2022
# Final Project
# Project Due: Thursday December 8, 2022 at 6:00 PM
## Proposals Due: Friday November 18, 2022 at 6:00 PM


You have two options for completing your final project for this course. 

#### Option 1 ####
The first option is presented in this notebook and involves implementing a Connect Four game with AB pruning and A* as player strategies. 

#### Option 2 ####
The second option is to design your own project that includes any of the algorithms we've discussed throughout the semester, or an algorithm that you're interested in learning that we haven't discussed in class. Your project also needs to include some kind of analysis of how it performed on a specific problem. If you're interested in the design your own project option, you need to discuss your idea with one of the course instructors to get approval. If you do a project without getting approval, you will receive a 0 regardless of the quality of the project. 

**The rules:**

1. Choose EITHER the given problem to submit OR choose your own project topic. 

2. If you choose your own project topic, please adhere to the following guidelines:
- Send an email to the course instructors before Friday, November 18 at 6pm, with a paragraph description of your project. We will respond within 24 hours with feedback.
- The project can include an algorithm we've discussed throughout the semester or an algorithm that you're been curious to learn. Please don't recycle a project that you did in another class. 
- If you do your own project without prior approval, you will receive a 0 for this project.
- Your project code, explanation, and results must all be contained in a Jupyter notebook. 

3. All work, code and analysis must be **your own**.
4. You may use your course notes, posted lecture slides, textbook, in-class notebooks and homework solutions as resources.  You may also search online for answers to general knowledge questions, like the form of a probability distribution function, or how to perform a particular operation in Python. You may not use entire segments of code as solutions to any part of this project, e.g. if you find a Python implementation of policy iteration online, you can't use it.
5. You may **not** post to message boards or other online resources asking for help.
6. **You may not collaborate with classmates or anyone else.**
7. This is meant to be like a coding portion of an exam. So, we will be much less helpful than we typically are with homework. For example, we will not check answers, help debug your code, and so on.
8. If you have a question, post it first as a **private** Piazza message. If we decide that it is appropriate for the entire class, then we will make it a public post (and anonymous).
9. If any part of the given project or your personal project is left open-ended, it is because we intend for you to code it up however you want. Our primary concern is with the plots/analysis that your code produces. Feel free to ask clarifying questions though.

Violation of these rules will result in an **F** and a trip to the Honor Code council.

---
**By writing your name below, you agree to abide by these rules:**

**Your name:** Ty VanEssen

---


---

Some useful packages and libraries:



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from collections import deque
import heapq
import unittest
from scipy import stats
import copy as cp
from time import time

---

## Problem 1: Game Theory - Playing "intelligent" Connect Four

Connect Four is a two-player game where the objective is to get four pieces in a row - horizontally, vertically, or diagonally. Check out this video if you're unfamiliar with the game: https://www.youtube.com/watch?v=utXzIFEVPjA.

### (1a)   Defining the Connect Four class structure

We've provided the humble beginnings of a Connect Four game. You need to fill in this class structure for Connect Four using what we did during class as a guide, and then implement min-max search with AB pruning, and A* search with at least one heuristic function. The class structure includes the following:

* `moves` is a list of columns to represent which moves are available. Recall that we are using matrix notation for this, where the upper-left corner of the board, for example, is represented at (1,1).
* `result(self, move, state)` returns a ***hypothetical*** resulting `State` object if `move` is made when the game is in the current `state`. Note that when a 'move' is made, the column must have an open slot and the piece must drop to the lowest row. 
* `compute_utility(self, move, state)` calculates the utility of `state` that would result if `move` is made when the game is in the current `state`. This is where you want to check to see if anyone has gotten `nwin` in a row
* `game_over(self, state)` returns `True` if the game in the given `state` has reached a terminal state, and `False` otherwise.
* `utility(self, state, player)` returns the utility of the current state if the player is Red and $-1 \cdot$ utility if the player is Black.
* `display(self)` is a method to display the current game `state`. You get it for free because this would be super frustrating without it.
* `play_game(self, player1, player2)` returns an integer that is the utility of the outcome of the game (+1 if Red wins, 0 if draw, -1 if Black wins). `player1` and `player2` are functional arguments that we will deal with in parts **1b** and **1d**.

Some notes:
* Assume Red always goes first.
* Do **not** hard-code for 6x7 boards.
* You may add attributes and methods to these classes as needed for this problem.

In [2]:

from enum import Enum
import sys
from typing import Callable

class Player(Enum):
    RED = "R"
    BLACK = "B"
    EMPTY = "."

class State:
    def __init__(self, board_dimensions: tuple[int, int], utility: int = 0, board: dict[tuple[int,int], Player] = {}):
        self.to_move = Player.RED
        self.utility = utility
        self.dimensions = board_dimensions
        self.board : dict[tuple[int,int], Player] = board
        self._moves = None
        self._adjacent = None

    def moves(self) -> list[tuple[int, int]]:
        # Lazy Cache
        if (self._moves != None): 
            return self._moves

        available_moves : dict[int, int] = {col: self.dimensions[0] for col in range(1, self.dimensions[1] + 1)}
        for taken_row, taken_col in self.board:
            if taken_col in available_moves:
                next_row = taken_row - 1
                if next_row <= 0:
                    available_moves.pop(taken_col)
                elif next_row < available_moves[taken_col]:
                    available_moves[taken_col] = next_row
        self._moves = [ (available_moves[col], col) for col in available_moves]
        return self._moves


    def get_adjacent(self, move: tuple[int,int], n_adjacent: int, search_for: list[Player] = [*Player]) -> list[tuple[list[Player], list[Player]]]:
        if Player.EMPTY in search_for and not None in search_for:
            search_for.append(None)
        check_directions = [
            (iter_row, iter_col)
            for iter_row in range(-1,2)
            for iter_col in range(-1,2)
            if (iter_row, iter_col) != (0,0)
        ]
        surrounding: dict[tuple[int,int], list[Player]] = {
            direction: [] for direction in check_directions
        }

        for levels_out in range(1,n_adjacent):
            for direction in check_directions.copy():
                check_state = (move[0] + direction[0] * levels_out, move[1] + direction[1] * levels_out)
                
                if not (check_state[0] in range(1, self.dimensions[0] + 1) and check_state[1] in range(1, self.dimensions[1] + 1)):
                    continue

                check_state_value : Player | None = self.board.get(check_state)

                if check_state_value in search_for:
                    value = Player.EMPTY if check_state_value == None else check_state_value
                    surrounding[direction].append(value)
                else:
                    check_directions.remove(direction)
        
        sequential: list[tuple[list[Player], list[Player]]] = []
        directions = list(surrounding.keys())
        for direction_idx in range(int(len(directions) / 2)):
            f_dir = directions[direction_idx]
            b_dir = directions[len(directions) - direction_idx - 1]
            sequential.append((surrounding[f_dir], surrounding[b_dir]))
        assert len(sequential) == 4
        return sequential
    
    def hash(self):
        return "~".join([str(e)+ ":" + str(self.board[e]) for e in self.board])

class ConnectFour:
    def __init__(self, nrow=6, ncol=7, nwin=4):
        self.nrow: int = nrow
        self.ncol: int = ncol
        self.nwin: int = nwin
        self.state = State((nrow, ncol))

    def result(self, move: tuple[int,int], state: State) -> State:
        '''
        What is the hypothetical result of move `move` in state `state` ?
        move  = (row, col) tuple where player will put their mark (R or B)
        state = a `State` object, to represent whose turn it is and form
                the basis for generating a **hypothetical** updated state
                that will result from making the given `move`
        '''
        
        # your code goes here...
        if not move in state.moves():
            raise ValueError('Move {} is not in moves {}'.format(move, state.moves()))
        #? Should this be +=
        utility = self.compute_utility(move, state)
        new_state = State((self.nrow, self.ncol), utility, {**state.board, move: state.to_move})
        new_state.to_move = Player.BLACK if state.to_move == Player.RED else Player.RED

        return new_state

    def compute_utility(self, move: tuple[int,int], state: State) -> int:
        '''
        What is the utility of making move `move` in state `state`?
        If 'R' wins with this move, return 1;
        if 'B' wins return -1;
        else return 0.
        '''        
        
        # your code goes here...
        adjacent = state.get_adjacent(move, self.nwin, [state.to_move])
        for angle in adjacent:
            sequential = 1
            for direction in angle:
                for piece in direction:
                    if piece == state.to_move:
                        sequential += 1
                    else:
                        break
            if sequential >= self.nwin:
                return 1 if state.to_move == Player.RED else -1
        return 0

    def game_over(self, state: State) -> bool:
        '''game is over if someone has won (utility!=0) or there
        are no more moves left'''

        # your code goes here... 
        return state.utility != 0 or len(state.moves()) == 0

    def utility(self, state: State, player: Player) -> int:
        '''Return the value to player; 1 for win, -1 for loss, 0 otherwise.'''
        # your code goes here...
        #? This should be a toggle that allows for multi 4match completes
        red_utility: int = -1 if state.utility < 0 else int(state.utility > 0)
        return red_utility if player == Player.RED else -1 * red_utility

    def display(self, state: State = None) -> None:
        board = state.board if state != None else self.state.board

        for row in range(1, self.nrow + 1):
            for col in range(1, self.ncol + 1):
                print(board.get((row, col), Player.EMPTY).value, end=' ')
            print()

    def play_game(self, player1, player2, verbose: bool = False) -> int:
        '''Play a game of Connect Four!'''
        
        # your code goes here...
        player_actors: dict[Player, Callable[ConnectFour, tuple[int,int]]] = {
            Player.RED: player1,
            Player.BLACK: player2
        }

        while not self.game_over(self.state):
            actor = player_actors[self.state.to_move]
            move: tuple[int,int] = actor(self)
            new_state = self.result(move, self.state)
        
            relative_utility = self.utility(new_state, self.state.to_move)

            if verbose: print("{} places a piece at {}".format(self.state.to_move.name, move))
            if relative_utility != 0 and verbose:
                print("Player {} Won!".format(self.state.to_move.name if relative_utility > 0 else new_state.to_move.name))
            self.state = new_state
            
            if verbose: self.display()
            sys.stdout.flush()

        
        return self.utility(self.state, Player.RED)

### (1b) Define a random player

Define a function `random_player` that takes a single argument of the `ConnectFour` class and returns a random move out of the available legal moves in the `state` of the `ConnectFour` game.

In your code for the `play_game` method above, make sure that `random_player` could be a viable input for the `player1` and/or `player2` arguments.

In [3]:
def random_player(game: ConnectFour) -> tuple[int,int]:
    '''A player that chooses a legal move at random out of all
    available legal moves in ConnectFour state argument'''
    
    # your code goes here...
    avaliable_moves = game.state.moves()
    selected_move : int = np.random.choice(range(len(avaliable_moves)))
    return avaliable_moves[selected_move]

We know from experience and/or because I'm telling you right now that if two `random_player`s play many games of ConnectFour against one another, whoever goes first will win about 55% of the time.  Verify that this is the case by playing at least 1,000 games between two random players. Report the proportion of the games that the first player has won.**(Chris: is this true for TicTacToe, or Connect Four)**

**"Unit tests":** If you are wondering how close is close enough to 55%, I simulated 100 tournaments of 1,000 games each. The min-max range of win percentage by the first player was 52-59%.

In [4]:
# 1000 games between two random players

# Your code here
def run_games(player1, player2, iterations: int, game_args = (6,7,4), return_games = False):
    winners : dict[Player, int] = {p: 0 for p in Player}
    games = []
    for i in range(iterations):
        game = ConnectFour(*game_args)
        games.append(game)
        game_result = game.play_game(player1, player2)
        winner = Player.EMPTY
        if game_result < 0:
            winner = Player.BLACK
        elif game_result > 0:
            winner = Player.RED

        winners[winner] += 1
    print("Player {} won {:.4f}% of the time".format(Player.RED.name, 100 * winners[Player.RED] / iterations))
    print("Player {} won {:.4f}% of the time".format(Player.BLACK.name, 100 * winners[Player.BLACK] / iterations))
    print("Tied {:.4f}% of the time".format(100 * winners[Player.EMPTY] / iterations))

    if return_games:
        return games

In [5]:
run_games(random_player, random_player, 1000)

Player RED won 54.6000% of the time
Player BLACK won 45.2000% of the time
Tied 0.2000% of the time


### (1c) What about playing randomly on different-sized boards?

What does the long-term win percentage appear to be for the first player in a 10x10 ConnectFour tournament, where 4 marks must be connected for a win?  Support your answer using a simulation and printed output, similar to **1b**.

**Also:** The win percentage should have changed substantially. Did the decrease in wins turn into more losses for the first player or more draws? Write a few sentences explaining the behavior you observed.  *Hint: think about how the size of the state space has changed.*

In [6]:
# 1000 games between two random players
# Your code here
#Please also recall that you should be using a 10x10 with nwin as 6 for the 1c analysis. 
run_games(random_player, random_player, 1000, (10,10,6))

Player RED won 45.7000% of the time
Player BLACK won 46.1000% of the time
Tied 8.2000% of the time


The win percent did change significantly with a larger board and nwin value. This seems to be primarirly because the advantage for going first is mostly negated with a much larger state space. There are more moves available to the random player so it is less likely that they will chance on a winning solution. Most of the difference in win rates becomes tied matches between the players. It makes sense to have a high tie percentage since each of these players are acting randomly.

### (1d) Define an alpha-beta player

Alright. Let's finally get serious about our Connect Four game.  No more fooling around!

Craft a function called `alphabeta_player` that takes a single argument of a `ConnectFour` class object and returns the minimax move in the `state` of the `ConnectFour` game. As the name implies, this player should be implementing alpha-beta pruning as described in the textbook and lecture.

Note that your alpha-beta search for the minimax move should include function definitions for `max_value` and `min_value` (see the aggressively realistic pseudocode from the lecture slides).

In your code for the `play_game` method above, make sure that `alphabeta_player` could be a viable input for the `player1` and/or `player2` arguments.

In [7]:
# Your code here

# Regarding the AB Pruning runtime issues, please shrink the game size to a 3x4 board with nwin as 3. This applies to parts 1d, 1e, and the comparison of A* vs AB pruning comparison in part 2b. This should be the only necessary modification to the writeup to alleviate the runtime delays. Make sure that your random_player and A* players work on 6x7 boards with nwin as 4.  
from collections import namedtuple

nodes_expanded = 0
discount_factor = .9
UtilityMove = namedtuple("UtilityMove", ["utility", "move"])

def alphabeta_player(game: ConnectFour, report_nodes_expanded = False) -> tuple[int,int]:
    global nodes_expanded
    nodes_expanded = 0
    utility = alphabeta_value(game, game.state, float("-inf"), float("inf"))
    if report_nodes_expanded: return nodes_expanded
    return utility.move

def alphabeta_value(game: ConnectFour, state: State, alpha: int, beta: int) -> UtilityMove:
    global nodes_expanded
    nodes_expanded += 1
    if game.game_over(state):
        return UtilityMove(state.utility, None)

    beta_pointer, alpha_pointer = [beta], [alpha]

    if state.to_move == Player.RED:
        utility = UtilityMove(-np.inf, None)
        comparison_fn = max
        child_rechable_boundary = alpha_pointer
        reachable_bound = beta_pointer
        symbol = float.__gt__
        # symbol = float.__ge__
    else:
        utility = UtilityMove(np.inf, None)
        comparison_fn = min
        child_rechable_boundary = beta_pointer
        reachable_bound = alpha_pointer
        symbol = float.__lt__
        # symbol = float.__le__
    
    #! Room to mess with the order we evaluate moves, might improve runtime
    moves = state.moves()
    for move_idx in range(len(moves)):
        move = moves[move_idx]

        args = (game.result(move, state), alpha_pointer[0], beta_pointer[0])
        child_utility, child_move = alphabeta_value(game, *args)
        child_utility *= discount_factor
        #Dramatically improves performance by selecting for earlier wins

        utility = comparison_fn(utility, UtilityMove(child_utility, move))

        # Turns out that AB with a >=/<= will prune states that it shouldn't since they would still be winning for the other player evaluating with >/< will ensure that those states are found. The submitted code utilizes the optimal solution I found with >/< bound checking and discount factors.
        if symbol(float(utility.utility), float(reachable_bound[0])):
            return utility
        child_rechable_boundary[0] = comparison_fn(utility.utility, child_rechable_boundary[0])
    return utility

Verify that your alpha-beta player code is working appropriately through the following tests, using a standard 6x7 ConnectFour board. Run **10 games for each test**, and track the number of wins, draws and losses. Report these results for each case.

1. An alpha-beta player who plays first should never lose to a random player who plays second.
2. Two alpha-beta players should always draw. One player is the max player and the other player is the min player.

**Nota bene:** Test your code with fewer games between the players to start, because the alpha-beta player will require substantially more compute time than the random player.  This is why I only ask for 10 games, which still might take a minute or two. You are welcome to run more than 10 tests if you'd like. 

In [8]:
# Your code here
iters = 10
print("Alphabeta (RED) vs Random (BLACK)")
run_games(alphabeta_player, random_player, iters, (3,4,3))

print("Alphabeta (RED) vs Alphabeta (BLACK)")
run_games(alphabeta_player, alphabeta_player, iters, (3,4,3))

print("Random (RED) vs Alphabeta (BLACK)")
run_games(random_player, alphabeta_player, iters, (3,4,3))

Alphabeta (RED) vs Random (BLACK)
Player RED won 100.0000% of the time
Player BLACK won 0.0000% of the time
Tied 0.0000% of the time
Alphabeta (RED) vs Alphabeta (BLACK)
Player RED won 100.0000% of the time
Player BLACK won 0.0000% of the time
Tied 0.0000% of the time
Random (RED) vs Alphabeta (BLACK)
Player RED won 40.0000% of the time
Player BLACK won 60.0000% of the time
Tied 0.0000% of the time


### (1e) What has pruning ever done for us?

Calculate the number of number of states expanded by the minimax algorithm, **with and without pruning**, to determine the minimax decision from the initial 6x7 ConnectFour board state.  This can be done in many ways, but writing out all the states by hand is **not** one of them (as you will find out!).

Then compute the percent savings that you get by using alpha-beta pruning. i.e. Compute $\frac{\text{Number of nodes expanded with pruning}}{\text{Number of nodes expanded with minimax}} $

Write a sentence or two, commenting on the difference in number of nodes expanded by each search.

In [9]:
# Your code here.
game = ConnectFour(3,4,3)
alphabeta_player(game, True)

12447

There are several differently performing versions of pruning, my favorite with the best win percentage is includes a discount factor and doesn't prune on equality. This method expands 11890 states.

Here are the states expanded for the versions I have, ranked by their performance winning against random, with the number of states expanded.
- Discount, Pruned Equality: 74.8%, 11715
- Discount, Not Pruned Equality: 81.6%, 11890
- No Discount, Pruned Equality: 48.4%, 1115
  - Class code version, notice that this performs the worst but does expand the fewest states.
- No Discount, Not Pruned Equality: 60.4%, 95428
- No Pruning: 66.4%, 289869

Provided Ideal AB-Pruning compared to minimax expands 0.3% of the nodes, my found ideal with Discount factors not pruning on equality expands 4.1% of the nodes. There is a clear relationship between the use of >= / <= or > / < equality for decisions around pruning the AB tree, in the base version this ends up pruning a lot of very similar states since our utilities are only -1,0,1 this version is least performant. By not pruning on eqality we check the brances that may be ties in some cases but still are valid wins for the other player. There also seems to be a direct relationship between the use of discount factors to differentiate between multiple winning solutions, this allows us to prune trees that would lead to a win or loss sooner or earlier, and combined with non-equality pruning gives the best results when random plays first.

### (2) A* Search

In Part II of this project, you need to implement a player strategy to employ A* Search in order to win at ConnectFour. To test your A* player, play 10 games against the random player and 10 games against the AB pruning player. 

Write an explanation of this strategy's implementation and performance in comparison to the random player and the AB Pruning player from **1d**.

A lot of the code that you wrote for the minimax player and the Connect Four game structure can be reused for the A* player. However, you will need to write a new utility function for A* that considers the path cost and heuristic cost.


### (2a) Define a heuristic function
Your A* player will need to use a heuristic function. You have two options for heurstics: research an existing heuristic for Connect Four, or games similar to Connect Four, and use that. Or, design your own heuristic. Write a one-paragraph description of the heuristic you're using, including a citation if you used an existing heuristic.

For my heuristic function, I chose to take inspiration from the internet, specifically https://roadtolarissa.com/connect-4-ai-how-it-works/, without reading much of the article I saw that it was a process of trying to find optimal moves to set up multiple game pieces in a row. So I decided to leverage the get_adjacent function I added to state objects. We collate the combination patterns (rows / columns / diagonals) and check for where valid moves could be played and see if we could possibly win on that row given the current state. We can then check the number of pieces needed for that win and report that as the heuristic, we do the same for blocks, limiting them to a value so we don't worry about blocking early. 

In [10]:
#Todo Finish the writing bit https://roadtolarissa.com/connect-4-ai-how-it-works/
def heuristic(game: ConnectFour, move: tuple[int,int], state: State) -> float:
    other_p = Player.BLACK if state.to_move == Player.RED else Player.RED
    self_p = state.to_move

    heuristics = []
    blocks = [[*front, other_p, *back] for front, back in state.get_adjacent(move, game.nwin, [Player.EMPTY, other_p])]
    wins = [[*front, self_p, *back] for front, back in state.get_adjacent(move, game.nwin, [Player.EMPTY, self_p])]
    for direction_idx in range(len(wins)):
        row_wins = wins[direction_idx]
        row_blocks = blocks[direction_idx]

        dist_win = game.nwin - row_wins.count(self_p) if len(row_wins) >= game.nwin else np.inf
        dist_block = game.nwin - row_blocks.count(other_p) if len(row_blocks) >= game.nwin else np.inf
        heuristics.append(min(dist_win, dist_block))

    heuristic = min(heuristics)
    return heuristic

In [11]:
class Frontier_PQ:
    ''' frontier class for uniform search, ordered by path cost '''
    def __init__(self, start: State, cost = 0) -> None:
        self.start : State = start
        self.states : dict[str, tuple[int, State, State]] = {start.hash():(cost, start, None)}
        self.q : list[tuple[int, str, str]] = [(cost,start.hash(),None)]

    def add(self, state: State, cost: int, parent: State):
        heapq.heappush(self.q, [cost, state.hash(), parent.hash()])

        self.states[state.hash()] = (cost, state, parent)
    
    def pop(self) -> tuple[int, State, State]:
        popped_entry = None
        while popped_entry == None:
            popped_entry = heapq.heappop(self.q)
            if popped_entry[0] == "REMOVED":
                print("FOUND REMOVED")
                popped_entry = None
                
        return self.states.pop(popped_entry[1])

    def replace(self, state: State, new_cost: int, new_parent: State):
        previous_cost, old_state, old_parent = self.states.get(state.hash())    

        for idx in range(len(self.q)):
            old_cost, node, old_parent = self.q[idx]
            if node == state.hash():
                q_idx = idx
                break

        tmp_task = self.q.pop(q_idx)
        tmp_task[0] = "REMOVED"
        
        heapq.heappush(self.q, [new_cost, state.hash(), new_parent.hash()])
        self.states[state.hash()] = (new_cost, state, new_parent)

In [12]:
def path(previous: dict[State, State], s): 
    if s is None:
        return []
    else:
        return path(previous, previous[s])+[s]


def a_star(game: ConnectFour, start: State, return_cost = False):
    frontier = Frontier_PQ(start)
    parent_tree : dict[State, State]= {start: None}
    final_state: State = start

    while len(frontier.states) != 0:
        previous_cost, current, parent = frontier.pop()
        parent_tree[current] = parent

        if game.game_over(current):
            final_state = current
            if game.utility(current, start.to_move) > 0:
                break

            no_chance_parents = [parent_tree[final_state]]
            # Stop exploring any branches where they could win in one move from ours.
            assert no_chance_parents[0].to_move != start.to_move, no_chance_parents[0].to_move + " != " + start.to_move.name
            def bad_children(bad_node: State, d = 0):
                for key, value in parent_tree.items():
                    if value == bad_node:
                        no_chance_parents.append(key)
                        bad_children(key, d + 1)
            bad_children(no_chance_parents[0])
            for st in frontier.states:
                f_cost, f_state, f_parent = frontier.states[st]
                if f_parent in no_chance_parents:
                    frontier.replace(f_state, np.inf, f_parent)
            continue
        
        for action in current.moves():
            child_state = game.result(action, current)
            child_path_cost = len(child_state.board) + heuristic(game, action, current)

            if not child_state in list(parent_tree.keys()) and not child_state in frontier.states:
                frontier.add(child_state, child_path_cost, current)
            elif child_state in frontier.states and frontier.states[child_state] > child_path_cost:
                frontier.replace(child_state, child_path_cost, current)

    solution_path: list[State] = path(parent_tree, final_state)
    solution_cost = len(solution_path)
    return (solution_path, solution_cost, len(parent_tree.keys())) if return_cost else solution_path

def a_star_player(game: ConnectFour, return_cost = False):
    sol_path : list[State]
    if return_cost:
        sol_path, sol_cost, explored = a_star(game, game.state, True)
    else:
        sol_path = a_star(game, game.state, False)

    for current_action in game.state.moves():
        result_state = game.result(current_action, game.state)
        if result_state.board == sol_path[1].board: return current_action
    return None


In [13]:
game = ConnectFour(4,5,4)
# game = ConnectFour()
# game.state = State((3,4), 0, {
#     (3,1): Player.RED,
#     (2,1): Player.RED,
#     (3,2): Player.BLACK,
#     (3,4): Player.BLACK
# })
# game.state.to_move = Player.BLACK
game.play_game(a_star_player, a_star_player, True)
# game.display()
# a = a_star_player(game)
# print("MOVE", a)

RED places a piece at (6, 1)
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
R . . . . . . 
BLACK places a piece at (5, 1)
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
B . . . . . . 
R . . . . . . 
RED places a piece at (6, 2)
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
B . . . . . . 
R R . . . . . 
BLACK places a piece at (4, 1)
. . . . . . . 
. . . . . . . 
. . . . . . . 
B . . . . . . 
B . . . . . . 
R R . . . . . 
RED places a piece at (6, 3)
. . . . . . . 
. . . . . . . 
. . . . . . . 
B . . . . . . 
B . . . . . . 
R R R . . . . 
BLACK places a piece at (3, 1)
. . . . . . . 
. . . . . . . 
B . . . . . . 
B . . . . . . 
B . . . . . . 
R R R . . . . 
RED places a piece at (6, 4)
Player RED Won!
. . . . . . . 
. . . . . . . 
B . . . . . . 
B . . . . . . 
B . . . . . . 
R R R R . . . 


1

### (2b) Compare A* to other algorithms
Next, play 10 games of Connect Four using your A* player and a random player and 10 games against the AB pruning player. In four or five paragraphs, report on the outcome. Did one player win more than the other? How often was the game a draw? How many moves did each player make? Were there situations where one player appeared to do better than the other? Given the outcome, are there other heuristics you would like to implement?

In [14]:
# Your code here.
def display_avg_moves(games: list[ConnectFour]):
    all_moves = {p:0 for p in Player}

    for game in games:
        game_board = game.state.board
        for p in game_board.values():
            all_moves[p] += 1

    for p in all_moves:
        print("Player {} made {} moves on average across {} games".format(
            p.name, all_moves[p]/len(games), len(games)
        ))

In [15]:
print("\tAstar vs Random, 6x7, 4 to win")
display_avg_moves(run_games(a_star_player, random_player, 10, return_games=True))

	Astar vs Random, 6x7, 4 to win
Player RED won 90.0000% of the time
Player BLACK won 10.0000% of the time
Tied 0.0000% of the time
Player RED made 4.9 moves on average across 10 games
Player BLACK made 4.0 moves on average across 10 games
Player EMPTY made 0.0 moves on average across 10 games


In [16]:
print("\tAstar vs Alphabeta, 3x4, 3 to win")
display_avg_moves(run_games(a_star_player, alphabeta_player, 10, (3,4,3), True))

	Astar vs Alphabeta, 3x4, 3 to win
Player RED won 100.0000% of the time
Player BLACK won 0.0000% of the time
Tied 0.0000% of the time
Player RED made 6.0 moves on average across 10 games
Player BLACK made 5.0 moves on average across 10 games
Player EMPTY made 0.0 moves on average across 10 games


In [17]:
print("\tAlphabeta vs Random, 3x4, 3 to win")
display_avg_moves(run_games(alphabeta_player, random_player, 10, (3,4,3), True))

	Alphabeta vs Random, 3x4, 3 to win
Player RED won 100.0000% of the time
Player BLACK won 0.0000% of the time
Tied 0.0000% of the time
Player RED made 3.4 moves on average across 10 games
Player BLACK made 2.4 moves on average across 10 games
Player EMPTY made 0.0 moves on average across 10 games


- Did one player win more than the other?
  - The A* player won the most, I put a lot of effort into making it's heuristics strong. In the test cases asked for by this question it will be most performant, I'm not supprised if it wins 100% of the time. AB will also beat out random with quite high frequency, although there will be ties. The A* algorithm is best, followed by AB-Pruning, followed by random. However random notably has a much higher win rate against AB-Pruning and has it's best win rates on smaller game boards. Random players almost never win on a large board.
- How often was the game a draw?
  - This also depends on the matchup and board size. But generally, the smaller boards and boards with a higher nwin will have a higher tie percentage since the winning states take up less of the total state space. AKA, it's harder to make matches when the board is small since other players can get in your way, and there is more need to do a blocking move. Ideally, A* would find mostly ties against itself, however it is optimizing on the shortness of the game rather than on the winning or not winning. 
  - A* versus A* should often result in a draw, however both players are optimizing for a short game, and the logic for searching out it's own winnning states could include a better method of assessing when to make a blocking move.
- How many moves did each player make? 
  - On average, these games were over in 5-6 moves when there were somewhat equally skillful opponents, when either AI played against random it was often over in little more than the nwin amount of turns. When we pair A* against AB-Pruning, oftentimes the length of the game is close to filling the entire board. 
- Were there situations where one player appeared to do better than the other?
  - There were a lot of situations where players had a distinct advantage over oneanother:
    - Random vs AB-Pruning on a small board, AB-Pruning is generally a very skillful opponent, however it assumes that it's opponent will act rationally. The Random player does anything but that, it manages to only let AB Pruning win ~70% of the time.
    - A* vs AB-Pruning, in this case A* will win out almost every time, this is because while AB-Pruning attempts to reason about states it focuses almost entirely on end game positions, however while A* reasons about states, it's looking at the reality of the board and attempting to both make winning moves and blocking moves. It still is gearing towards it's own win, but is also effectively blocking it's opponent.
- Given the outcome, are there other heuristics you would like to implement?
  - Although I put a lot of effort into tuning the various alorithms, I'd love to spend more time on AB-Pruning and come up with a new method of utility values to differentiate between the non-winning moves better. I think this would help it to find more tied games. I'd also love to spend more time (even though I already spent a lot) on the A* heuristic, currently when it plays against itself it struggles to reason about what the other player will do. My Ideal would be to use an A* framework to find the end states and then implement some sort of pruning on those states. This arises from messing with A* and realizing that while the heuristic guides it towards the end of the game, it doesn't have any ability to reason effectively about the nature of that win.